In [1]:
from flask import Flask, request
import pandas as pd
from flask import jsonify
from flask_cors import CORS

In [2]:
data = pd.read_parquet("recipes.parquet")

In [3]:
data

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38.0,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09 21:46:00+00:00,Make and share this Low-Fat Berry Blue Frozen ...,[https://img.sndimg.com/food/image/upload/w_55...,...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,None,"[Toss 2 cups berries with sugar., Let stand fo..."
1,39.0,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29 13:12:00+00:00,Make and share this Biryani recipe from Food.com.,[https://img.sndimg.com/food/image/upload/w_55...,...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,None,[Soak saffron in warm milk for 5 minutes and p...
2,40.0,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05 19:52:00+00:00,This is from one of my first Good House Keepi...,[https://img.sndimg.com/food/image/upload/w_55...,...,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,None,"[Into a 1 quart Jar with tight fitting lid, pu..."
3,41.0,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03 14:54:00+00:00,This dish is best prepared a day in advance to...,[https://img.sndimg.com/food/image/upload/w_55...,...,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"[Drain the tofu, carefully squeezing out exces..."
4,42.0,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19 06:19:00+00:00,Make and share this Cabbage Soup recipe from F...,[https://img.sndimg.com/food/image/upload/w_55...,...,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,None,"[Mix everything together and bring to a boil.,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522512,541379.0,Meg's Fresh Ginger Gingerbread,2002090414,rdsxc,PT35M,PT1H,PT1H35M,2020-12-22 15:27:00+00:00,Make and share this Meg's Fresh Ginger Gingerb...,[],...,7.6,54.4,278.2,48.5,0.8,22.8,3.9,8.0,1 8x8 cake pan,[Preheat oven to 350&deg;F Grease an 8x8 cake ...
522513,541380.0,Roast Prime Rib au Poivre with Mixed Peppercorns,211566,Denver cooks,PT3H,PT30M,PT3H30M,2020-12-22 15:32:00+00:00,"White, black, green, and pink peppercorns add ...",[https://img.sndimg.com/food/image/upload/w_55...,...,71.4,433.8,766.3,3.2,0.7,0.1,117.0,8.0,1 Roast,[Position rack in center of oven and preheat t...
522514,541381.0,Kirshwasser Ice Cream,2001131545,Jonathan F.,PT3H,PT1H,PT4H,2020-12-22 15:33:00+00:00,Make and share this Kirshwasser Ice Cream reci...,[],...,72.6,470.9,192.5,33.9,0.0,17.3,12.8,6.0,None,[heat half and half and heavy cream to a simme...
522515,541382.0,Quick & Easy Asian Cucumber Salmon Rolls,2001004241,CLUBFOODY,None,PT15M,PT15M,2020-12-22 22:11:00+00:00,"Extremely quick and easy to make, these are gr...",[https://img.sndimg.com/food/image/upload/w_55...,...,0.1,2.9,100.5,0.3,0.0,0.1,2.4,NaN,20 rolls,"[In a small bowl, combine mayo and wasabi past..."


In [4]:
data.loc[data['RecipeId']==38]

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38.0,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09 21:46:00+00:00,Make and share this Low-Fat Berry Blue Frozen ...,[https://img.sndimg.com/food/image/upload/w_55...,...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,None,"[Toss 2 cups berries with sugar., Let stand fo..."


In [9]:
import json

from elasticsearch import Elasticsearch


app = Flask(__name__)
CORS(app)

app.es_client = Elasticsearch("https://localhost:9200", basic_auth=("elastic","ScW45_K41iKePloVvMat"), ca_certs="~/http_ca.crt")
app.df = pd.read_parquet('no_na_recipes.parquet.gzip')


@app.route('/search', methods=['GET'])

def search_recipes():
    response_object = {'status': 'success'} 
    argList = request.args.to_dict(flat=False) 
    RecipeId = argList['RecipeId'][0] 
    
    result = data.loc[data['RecipeId']== int(RecipeId)]
    return json.loads(result.to_json(orient="records"))

@app.route('/search_recipes', methods=['GET'])
def search_recipess():
    response_object = {'status': 'success'}
    query_term = request.args.get('query')
    results = app.es_client.search(index='foodir', query={
            "script_score":{
                "query":{
                    "dis_max": {
                        "queries": [
                            { "match": { "Name": query_term }},
                            { "match": { "RecipeIngredientParts": query_term }},
                            { "match": { "RecipeInstructions": query_term }},
                        ],
                        "tie_breaker": 0.3
                    },
                },
                "script":{
                    "source": "_score * (((doc['ReviewCount'].value * doc['AggregatedRating'].value) + (3 * 32)) / (doc['ReviewCount'].value + 32 ))"
                }
            }
        }, suggest_field='Name', suggest_text=query_term, suggest_mode='missing', size=12)
    
    results_df = pd.DataFrame([[hit['_source'][key] for key in hit['_source']] for hit in results['hits']['hits']], columns=list(app.df.columns))
    results_df['_score'] = [hit['_score'] for hit in results['hits']['hits']]
    
    return results_df.to_dict(orient="records")

@app.route('/search_collection', methods=['GET'])
def search_collection():
    query_term = request.args.get('query')
    results = app.es_client.search(index='foodir', suggest_field='Name', suggest_text=query_term, suggest_mode='missing')
    return results['suggest']['Name']                  
    

if __name__ == '__main__':
    app.run(debug=False, port=4040)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:4040
Press CTRL+C to quit
127.0.0.1 - - [18/Mar/2024 22:23:14] "GET /search_collection?query=fist HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2024 22:48:36] "GET /search_recipes?query=bourbon%20ch HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2024 22:48:36] "GET /search_collection?query=bourbon%20ch HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2024 22:48:36] "GET /search_recipes?query=bourbon%20ch HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2024 23:14:55] "GET /search_recipes?query=bourbon%20ch HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2024 23:14:58] "GET /search_recipes?query=bourbon%20ch HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2024 23:14:58] "GET /search_collection?query=bourbon%20ch HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2024 23:15:16] "GET /search_recipes?query=bourbon%20ch HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2024 23:16:05] "GET /search_recipes?query=bourbon%20ch HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2024 23:16:08] "GET /search_recipes?query=bourbon%20ch HTTP/1.1" 200 -
127.0.0.1 - 

In [ ]:
data.columns

Index(['RecipeId', 'Name', 'AuthorId', 'AuthorName', 'CookTime', 'PrepTime',
       'TotalTime', 'DatePublished', 'Description', 'Images', 'RecipeCategory',
       'Keywords', 'RecipeIngredientQuantities', 'RecipeIngredientParts',
       'AggregatedRating', 'ReviewCount', 'Calories', 'FatContent',
       'SaturatedFatContent', 'CholesterolContent', 'SodiumContent',
       'CarbohydrateContent', 'FiberContent', 'SugarContent', 'ProteinContent',
       'RecipeServings', 'RecipeYield', 'RecipeInstructions'],
      dtype='object')